# **Mibac Sparql**

http://dati.beniculturali.it/sparql

es: Monastero di San Benedetto Sacro Speco
http://dati.beniculturali.it/lodview/mibact/luoghi/resource/CulturalInstituteOrSite/20359.html

Query esempio: http://dati.beniculturali.it/endpoint-sparql/query-di-esempio/


**SPARQL wrapper**

https://github.com/RDFLib/sparqlwrapper

https://rdflib.github.io/sparqlwrapper/



In [27]:
from SPARQLWrapper import SPARQLWrapper, JSON, XML 

In [28]:
ep = "http://dati.beniculturali.it/sparql"

In [29]:
sparql = SPARQLWrapper(ep)

In [33]:
q = """SELECT ?s ?nome ?provincia 
WHERE {
 ?s rdf:type cis:HolderOfArchive .
 ?s rdfs:label ?nome .
 ?s cis:hasSite ?location .
 ?location cis:hasAddress ?location2 .
 ?location2 cis:adminUnitL2 ?provincia .
 FILTER( (?provincia= "Milano") || 
 (?provincia= "Brescia") || 
 (?provincia= "Bergamo") || 
 (?provincia= "Como") || 
 (?provincia= "Cremona") || 
 (?provincia= "Lecco") || 
 (?provincia= "Lodi") ||
 (?provincia= "Pavia") ||
 (?provincia= "Mantova") ||
 (?provincia= "Sondrio") || 
 (?provincia= "Varese") || 
 (?provincia= "Monza e della Brianza"))
}
ORDER BY ?provincia """


In [56]:
q = '''select * where {

select distinct ?s as ?subject

?Nome_Istituzionale
?Descrizione
?ISILIdentifier
?Latitudine
?Longitudine
?Disciplina
?Indirizzo
?Codice_postale
?Comune
?Provincia
where {

graph <http://dati.beniculturali.it/mibact/luoghi> {

?s rdf:type cis:CulturalInstituteOrSite ;
cis:institutionalName ?Nome_Istituzionale .
optional { ?s cis:description ?Descrizione }
optional { ?s cis:ISILIdentifier ?ISILIdentifier }
optional { ?s geo:lat ?Latitudine }
optional { ?s geo:long ?Longitudine }
optional { ?s cis:hasDiscipline [cis:name ?Disciplina] }
optional {
?s cis:hasSite [cis:hasAddress ?address ] .
optional { ?address cis:fullAddress ?Indirizzo   }
optional { ?address cis:postCode ?Codice_postale }
optional { ?address cis:postName ?Comune         }
optional { ?address cis:adminUnitL2 ?Provincia   }
}
}
order by ?s
}
limit 5
offset 0
'''

In [54]:
#filter con regex
q = '''
PREFIX cis: <http://dati.beniculturali.it/cis/>
SELECT ?nome ?identifier
WHERE {
  ?x a cis:CulturalInstituteOrSite ;
  cis:identifier ?identifier ;
  cis:institutionalName ?nome .FILTER regex(str(?nome), "Sacro Speco") .
}
'''

In [58]:
#filter con contains
q = '''
PREFIX cis: <http://dati.beniculturali.it/cis/>
SELECT distinct ?nome ?identifier ?ind ?com
WHERE {
 ?x a cis:CulturalInstituteOrSite ;
  cis:institutionalName ?nome ;
  cis:identifier ?identifier .
  filter contains(?nome, "Racconigi"@it) 
}  
'''

In [60]:
q_complete= '''
construct {?s ?p ?o .} where { ?s ?p ?o . {
SELECT DISTINCT ?s WHERE {
 ?s rdf:type ?t .
 ?s rdf:type <http://dati.beniculturali.it/cis/CulturalInstituteOrSite>
 {?s ?p ?o0 . FILTER (bif:contains(?o0, '"ABBAZIA"'))}
 {?s ?p ?o1 . FILTER (bif:contains(?o1, '"DI"'))}
 {?s ?p ?o2 . FILTER (bif:contains(?o2, '"CASAMARI"'))}
}
LIMIT 24
OFFSET 0
} filter(?p=foaf:depiction || ?p=dc:isReferencedBy || ?p=cis:url || ?p=rdfs:label || ?p=skos:prefLabel || ?p=dc:title || ?p=dct:title || ?p=cis:identifier || ?p=cis:institutionalName) } LIMIT 1000
'''

In [62]:
#XML
sparql.setQuery(q)
sparql.setReturnFormat(XML)
results = sparql.query().convert()
print(results.toxml())


AttributeError: 'ConjunctiveGraph' object has no attribute 'toxml'

In [63]:
#JSON
sparql.setQuery(q)
sparql.setReturnFormat(JSON)
results = sparql.query().convert()

for result in results["results"]["bindings"]:
    print(result["nome"]["value"])
    print(result["identifier"]["value"])

/Users/mariaclaudiabodino/anaconda3/envs/py35/lib/python3.5/site-packages/SPARQLWrapper/Wrapper.py:684: RuntimeWarning: Sending Accept header '*/*' because unexpected returned format 'json' in a 'CONSTRUCT' SPARQL query form
  warnings.warn("Sending Accept header '*/*' because unexpected returned format '%s' in a '%s' SPARQL query form" % (self.returnFormat, self.queryType), RuntimeWarning)


KeyError: 'nome'

In [65]:
#JSON
sparql.setQuery(q_complete)
sparql.setReturnFormat(JSON)
results = sparql.query().convert()

print (results)

{'head': {'link': [], 'vars': ['s', 'p', 'o']}, 'results': {'ordered': True, 'bindings': [{'o': {'lang': 'it', 'type': 'literal', 'value': 'Abbazia di Casamari'}, 's': {'type': 'uri', 'value': 'http://dati.beniculturali.it/mibact/luoghi/resource/CulturalInstituteOrSite/20754'}, 'p': {'type': 'uri', 'value': 'http://dati.beniculturali.it/cis/institutionalName'}}, {'o': {'lang': 'it', 'type': 'literal', 'value': "Museo dell'Abbazia di Casamari"}, 's': {'type': 'uri', 'value': 'http://dati.beniculturali.it/mibact/luoghi/resource/CulturalInstituteOrSite/101427'}, 'p': {'type': 'uri', 'value': 'http://dati.beniculturali.it/cis/institutionalName'}}, {'o': {'type': 'literal', 'value': 'DBUnico.112086'}, 's': {'type': 'uri', 'value': 'http://dati.beniculturali.it/mibact/luoghi/resource/CulturalInstituteOrSite/112086'}, 'p': {'type': 'uri', 'value': 'http://dati.beniculturali.it/cis/identifier'}}, {'o': {'lang': 'it', 'type': 'literal', 'value': 'Biblioteca statale del Monumento nazionale di Ca